Importing all the necessary packages

In [1]:
from bs4 import BeautifulSoup
import urllib
import bs4
import pandas as pd

This is my target URL

In [2]:
url = 'https://en.wikipedia.org/wiki/Super_Bowl_LIV'

Using BeautifulSoup to get the crawled site, and fetching the title through it

In [3]:
with urllib.request.urlopen(url, data=None) as response:
        soup = bs4.BeautifulSoup(response)
        titles=soup.select('h1.firstHeading')[0].text.strip()

Getting all the url's referenced by ' href' html tag and prints the number of such tags.
I'm also restricting the number of urls fetched to 500


In [4]:
href_urls = set()
for a_tag in soup.html.body.find_all('a', limit = 500):
    if 'href' in a_tag.attrs:
        href_urls.add(a_tag.attrs['href'])
print(len(href_urls))

390


Parsing the code to convert the reference URL's to absolute URL's

In [5]:
abs_urls = set()
for href_url in href_urls:
        a_url = urllib.parse.urljoin(url, href_url)
        abs_urls.add(a_url)

Separating the source URL into its parts and filtering the URL's belonging to a particular netloc

In [6]:
parsed_source = urllib.parse.urlparse(url)
print(parsed_source.netloc)
wiki_urls = set()
for a_url in abs_urls:
    parsed_a_url = urllib.parse.urlparse(a_url)
    if parsed_a_url.netloc == parsed_source.netloc:
        wiki_urls.add(a_url)

en.wikipedia.org


cleaning the URL's to remove all irrelavent ones

In [7]:
cleaned_urls = set()
for a_url in wiki_urls:
    defrag_url = urllib.parse.urldefrag(a_url)
    cleaned_urls.add(defrag_url.url)

Creating 3 list 

 - 'webpage_url_source', where the first element is 'None' and appending the source url to the rest of the 99 URL's
 - 'webpage_url_list', where the first element is the wikipedia entry url
 - 'webpage_url_title', where the element is the wiki entry page title 


In [8]:
webpage_url_source = []
webpage_url_source.append('None')

for i in range(1,100):
    webpage_url_source.append(url)

webpage_url_list = [url]
webpage_url_title = [titles]

This cell - 
- loops through 100 url's
- filters the pages which is of the 'text/html' format
- sends the request through soup
- scrapes the title of the page
- prints them

In [9]:
%%time
i = 0
for to_open_url in cleaned_urls:
    try:
        if( i > 99): 
            break
        else:
            with urllib.request.urlopen(to_open_url, data=None, timeout=5) as response:
                if 'text/html' in response.headers['Content-Type']:
                    webpage_url_list.append(to_open_url)
                    with urllib.request.urlopen(to_open_url, data=None, timeout = 5) as response_title:
                        soup = bs4.BeautifulSoup(response_title)
                        titles=soup.select('h1.firstHeading')[0].text.strip()
                        webpage_url_title.append(titles)
                        print(titles)
                    i = i + 1
    except:
        pass 


1966 Kansas City Chiefs season
Baltimore Ravens
Germany
2019 Stanley Cup Finals
Central Time Zone
Donald Trump 2020 presidential campaign
American football
Broadcasting Act (1991)
Super Bowl Experience
Super Bowl IV
New England Patriots
Raymond James Stadium
Matt Moore (American football)
2020 Democratic Party presidential primaries
Iowa caucuses
Glossary of American football
Dee Ford
2019 Tennessee Titans season
Super Bowl XXXIII
Yolanda Adams
Pepsi
CTV Television Network
Ultra-high-definition television
TV6 (Sweden)
2019 Minnesota Vikings season
Verizon Media
2012 Baltimore Ravens season
Super Bowl XXXII
Eastern Time Zone
Play from scrimmage
St. Louis Cardinals
Damien Williams
South Beach
Super Bowl LV
Red zone (gridiron football)
Ocean Drive (South Beach)
DeForest Buckner
Major professional sports leagues in the United States and Canada
Fox Sports 1
Hard Rock Stadium
NFL on Westwood One Sports
beIN Sports
SoFi Stadium
Anthony Hitchens
Tyreek Hill
Quarterback
Ferris wheel
Frank Clark

this cell 
- zips the three lists ( source list, url list and title list)
- converts into a pandas dataframe and gives it a column names
- Adds " to the dataframe so that we don't get wrong information during the csv export
- downloads the data as csv

In [14]:
import csv
list = zip(webpage_url_source,webpage_url_list,webpage_url_title)
df = pd.DataFrame(list, columns=["url_source", "url_target", "page_title_target"])
df = '"'+df+'"'
df.to_csv('crawl.csv', index=False)